In [1]:
import os
from cot import Collection
from cot.generate import FRAGMENTS
from rich.pretty import pprint
import json
from sprint_utils import get_answer_extraction, join_strings

In [2]:
# "commonsense_qa", "validation"
# "med_qa", "test"
# "medmc_qa", "validation"
# "open_book_qa", "test"
# "strategy_qa", "train"
# "worldtree", "test"

In [3]:
# None,         # no cot-trigger
# 'kojima-01',  # Answer: Let's think step by step.
# 'kojima-02',  # Answer: First,
# 'kojima-03',  # Answer: Let's think about this logically.

In [4]:
def model_sprint_thoughtsource(collection, dataset_name_splits, api_service_model, cot_trigger_key=None):
    # collection = create_thoughtsource_100()
    collection = collection
    for dataset_name, split in dataset_name_splits:
        print(dataset_name)
        answer_extraction_key = get_answer_extraction(collection[dataset_name][split][0]["type"], collection[dataset_name][split][0]["choices"])
        for api_service, model, api_time_interval in api_service_model:

            config={
                "instruction_keys": None,
                "cot_trigger_keys": cot_trigger_key,
                "answer_extraction_keys": [answer_extraction_key], # Therefore, among A through C/D/E/F, the answer is'
                "author" : "thoughtsource",
                "api_service": api_service,
                "engine": model,
                "temperature": 0,
                "max_tokens": 512,
                "api_time_interval": api_time_interval,
                "verbose": False,
                "warn": False,
            }
        
            collection.generate(name=dataset_name, split=split,config=config)
            collection.dump("extra_save_thoughtsource_100_" + dataset_name + "_huggingface_endpoint.json")
    collection.evaluate()
    # write model name manually in here, or it gives an error because of the "/" in the model name = endpoint url in this case
    collection.dump("thoughtsource_100" + "_" + api_service + "_" + "google_flan-t5-xxl" + join_strings(config["cot_trigger_keys"]) + ".json")

In [5]:
# ENDPOINT TEST
endpoint_url = (
    "https://vg66yz1taapw6ef6.us-east-1.aws.endpoints.huggingface.cloud"
)
number_examples = 1
dataset_name_splits = [("commonsense_qa", "validation")] 

# api_service_model = [("huggingface_hub", "google/flan-t5-small", 1)]
cot_trigger_keys = ["kojima-01"]

In [6]:
dataset_name_splits = [("commonsense_qa", "validation"),
                        # ("med_qa", "test"),
                        # ("medmc_qa", "validation"),
                        # ("open_book_qa", "test"),
                        # ("strategy_qa", "train"),
                        # ("worldtree", "test"),
                        ]

# api_service_model = [("huggingface_hub", "google/flan-t5-xl", 1)]
api_service_model = [("huggingface_endpoint", endpoint_url, 0)]
# api_service_model = [("mock_api", "mock_model", 0)]
cot_trigger_key = [None, "kojima-01"]

In [7]:
collection = Collection.from_json("sprint_data/thoughtsource_1_empty.json")
# collection = Collection.from_json("sprint_data/thoughtsource_100_empty.json")

[nltk_data] Downloading package punkt to /home/kon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
model_sprint_thoughtsource(collection, dataset_name_splits, api_service_model, cot_trigger_key)

In [ ]:
eval = collection.evaluate()
pprint(eval)